<a href="https://colab.research.google.com/github/jonlwowski012/StockSellRulesChecker/blob/master/StockScreener/Screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
! git clone https://github.com/jonlwowski012/StockSellRulesChecker.git
! pip install -r StockSellRulesChecker/StockScreener/requirements.txt
! pip install -U git+https://github.com/mariostoev/finviz
! pip install "tqdm==4.43.0"

Cloning into 'StockSellRulesChecker'...
remote: Enumerating objects: 96, done.
remote: Counting objects: 100% (96/96), done.
remote: Compressing objects: 100% (87/87), done.
remote: Total 96 (delta 27), reused 26 (delta 6), pack-reused 0
Unpacking objects: 100% (96/96), done.
     |████████████████████████████████| 9.5MB 5.1MB/s 
     |████████████████████████████████| 1.3MB 48.0MB/s 
     |████████████████████████████████| 143kB 34.8MB/s 
     |████████████████████████████████| 296kB 43.2MB/s 
  Created wheel for finviz: filename=finviz-1.3.4-cp37-none-any.whl size=15193 sha256=e8b2c79fc9a89c836f16935114b5e4729cb8e8e11549efebc54c92e3c6be0732
  Stored in directory: /root/.cache/pip/wheels/6d/fa/7c/4cc075d8af3f8ed083dd112b1a2b09888b9a9a2964c907291f
  Created wheel for stockquotes: filename=stockquotes-2.0.1-cp37-none-any.whl size=4472 sha256=68053df538da9fd1e2fa906c1c2f55f66970c2c9293b4083f5413070e327193e
  Stored in directory: /root/.cache/pip/wheels/b6/5a/fa/c250a28690344108071fb9b467

  Cloning https://github.com/mariostoev/finviz to /tmp/pip-req-build-6r4gkvox
  Running command git clone -q https://github.com/mariostoev/finviz /tmp/pip-req-build-6r4gkvox
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for finviz: filename=finviz-1.3.4-cp37-none-any.whl size=16979 sha256=81dcda108ba8f8465a130bc0dadf655488888d904cb89dcea8402245bd0d0335
  Stored in directory: /tmp/pip-ephem-wheel-cache-akson8f8/wheels/98/cc/6a/68893eb662f0f196f8e153e9ce4424a8ae9e1f022fd64e4d6d
Successfully built finviz
  Found existing installation: finviz 1.3.4
    Uninstalling finviz-1.3.4:
      Successfully uninstalled finviz-1.3.4
     |████████████████████████████████| 61kB 2.6MB/s 
  Found existing installation: tqdm 4.41.1
    Uninstalling tqdm-4.41.1:
      Successfully uninstalled tqdm-4.41.1


In [47]:
from finviz.screener import Screener
import nest_asyncio
import finviz
import pandas_datareader as pdr
import yfinance as yf
yf.pdr_override() # <== that's all it takes :-)

import datetime
import stockquotes
import pandas as pd
import tqdm
nest_asyncio.apply()

import time
from tqdm.contrib.concurrent import process_map


In [58]:
eps_5year_over0pct = True
eps_qoq_over0pct = True
eps_yoy_over0pct = True
sales_5years_over0pct = True
sales_qoq_over0pct = True

filters = []
if eps_5year_over5pct:
  filters.append('fa_eps5years_pos')
if eps_qoq_over5pct:
  filters.append('fa_epsqoq_pos')
if eps_yoy_over5pct:
  filters.append('fa_epsyoy_pos')
if sales_5years_over5pct:
  filters.append('fa_sales5years_pos')
if sales_qoq_over5pct:
  filters.append('fa_salesqoq_pos')

stock_list = Screener(filters=filters, table='Performance', order='price')  

In [59]:
print(len(stock_list.data))

1569


In [60]:

def moving_average(yahoo_df, days, delta=0):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max() - datetime.timedelta(days=delta)
    end_date = curr_date - datetime.timedelta(days=days) - datetime.timedelta(days=delta)
    
    df_days = df[df['Date'] >= end_date]
    df_days = df_days[df_days['Date'] <= curr_date]
    return float(df_days['Close'].mean())

def moving_average_volume(yahoo_df, days, delta=0):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max() - datetime.timedelta(days=delta)
    end_date = curr_date - datetime.timedelta(days=days) - datetime.timedelta(days=delta)
    
    df_days = df[df['Date'] >= end_date]
    df_days = df_days[df_days['Date'] <= curr_date]
    return float(df_days['Volume'].mean())

def week52_low_high(yahoo_df):
    df = yahoo_df.reset_index()
    curr_date = df['Date'].max()
    end_date = curr_date - datetime.timedelta(days=365)
    
    df_days = df[df['Date'] >= end_date]
    return float(df_days['Close'].max()), float(df_days['Close'].min())

def SMA200_slope_positive_rule(yahoo_df, days=21):
  for day in range(days):
    if moving_average(yahoo_df, days=200, delta=day) > moving_average(yahoo_df, days=200, delta=day+1):
      continue
    else:
      return False
  return True

def screen_stock(stock):
  if stock["Ticker"] == "":
    return

  finviz_stats = finviz.get_stock(stock['Ticker'])
  prev_close = float(finviz_stats['Prev Close'].replace("$",""))

  # Price greater than $10 rule
  if prev_close < 10:
    return

  screened_stocks[stock['Ticker']] = {}
  
  try:
      yahoo_df = pdr.get_data_yahoo(stock['Ticker'], interval = "d")
  except:
    return

  SMA200_value = moving_average(yahoo_df, days=200)
  SMA150_value = moving_average(yahoo_df, days=150)
  SMA50_value = moving_average(yahoo_df, days=50)
  SMA50_volume_value = moving_average_volume(yahoo_df, days=50)

  # Liquidity Rule
  if SMA50_value*SMA50_volume_value <= 20e6:
    return

  SMA200_percent = float(finviz_stats['SMA200'].replace("%",""))/100
  SMA50_percent = float(finviz_stats['SMA50'].replace("%",""))/100
  volume = float(finviz_stats['Volume'].replace(",",""))

  try:
    EPS_QoQ_percent = float(finviz_stats['EPS Q/Q'].replace("%",""))/100
  except:
    EPS_QoQ_percent = 0

  try:
    Sales_QoQ_percent = float(finviz_stats['Sales Q/Q'].replace("%",""))/100
  except:
    Sales_QoQ_percent = 0

  try:
    inst_own = float(finviz_stats['Inst Own'].replace("%",""))/100
  except:
    inst_own = 0
  
  shares_outstanding = finviz_stats['Shs Outstand']
  if 'M' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("M", ""))*1e6
  elif 'B' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("B", ""))*1e9
  elif 'T' in shares_outstanding:
    shares_outstanding = float(shares_outstanding.replace("T", ""))*1e12
  else:
    shares_outstanding = 0

  week52_high, week52_low = week52_low_high(yahoo_df)
  
  screened_stocks[stock['Ticker']]['SMA200_value'] = SMA200_value
  screened_stocks[stock['Ticker']]['SMA150_value'] = SMA150_value
  screened_stocks[stock['Ticker']]['SMA50_value'] = SMA50_value
  screened_stocks[stock['Ticker']]['SMA200_percent'] = SMA200_percent
  screened_stocks[stock['Ticker']]['SMA50_percent'] = SMA50_percent
  screened_stocks[stock['Ticker']]['EPS_QoQ_percent'] = EPS_QoQ_percent
  screened_stocks[stock['Ticker']]['Sales_QoQ_percent'] = Sales_QoQ_percent
  screened_stocks[stock['Ticker']]['prev_close'] = prev_close
  screened_stocks[stock['Ticker']]['week52_high'] = week52_high
  screened_stocks[stock['Ticker']]['week52_low'] = week52_low
  screened_stocks[stock['Ticker']]['Inst. Ownership'] = inst_own
  screened_stocks[stock['Ticker']]['Shares Outstanding'] = shares_outstanding
  screened_stocks[stock['Ticker']]['volume'] = volume

  # volume*price > $15mm rule
  if volume*prev_close > 15000000:
    vol_price_rule = True
  else:
    vol_price_rule = False
  screened_stocks[stock['Ticker']]['vol_price_rule'] = vol_price_rule

  # EPS QoQ Yearly > 18% rule
  if EPS_QoQ_percent >= .18:
    eps_QoQ_yearly_rule = True
  else:
    eps_QoQ_yearly_rule = False
  screened_stocks[stock['Ticker']]['eps_QoQ_yearly_rule'] = eps_QoQ_yearly_rule

  # Sales QoQ Yearly > 25% rule
  if Sales_QoQ_percent >= .25:
    sales_QoQ_yearly_rule = True
  else:
    sales_QoQ_yearly_rule = False
  screened_stocks[stock['Ticker']]['sales_QoQ_yearly_rule'] = sales_QoQ_yearly_rule

  # Shares Outstanding <= 25 mil
  if shares_outstanding <= 25e6:
    shares_outstanding_rule = True
  else:
    shares_outstanding_rule = False
  screened_stocks[stock['Ticker']]['shares_outstanding_rule'] = shares_outstanding_rule

  # Institutional Ownership < 35%
  if 0.05 <= inst_own <= .35:
    inst_ownership_rule = True
  else:
    inst_ownership_rule = False
  screened_stocks[stock['Ticker']]['inst_ownership_rule'] = inst_ownership_rule

  # Positive 200d MA positive
  SMA200_slope_rule = SMA200_slope_positive_rule(yahoo_df, days=21)
  screened_stocks[stock['Ticker']]['SMA200_slope_rule'] = SMA200_slope_rule
      
  # 150d MA greater than 200d MA
  if SMA150_value > SMA200_value:
      SMA150_greater_SMA200_rule = True
  else:
      SMA150_greater_SMA200_rule = False
  screened_stocks[stock['Ticker']]['SMA150_greater_SMA200_rule'] = SMA150_greater_SMA200_rule
      
  # 50d MA greater than 150d MA
  if SMA50_value > SMA150_value:
      SMA50_greater_SMA150_rule = True
  else:
      SMA50_greater_SMA150_rule = False
  screened_stocks[stock['Ticker']]['SMA50_greater_SMA150_rule'] = SMA50_greater_SMA150_rule
      
  # Close above 50d MA
  if prev_close > SMA50_value:
      close_greater_SMA50_rule = True
  else:
      close_greater_SMA50_rule = False
  screened_stocks[stock['Ticker']]['close_greater_SMA50_rule'] = close_greater_SMA50_rule
      
  # 52 week high low span rule
  if 0.75*week52_high > 1.25*week52_low:
      week52_span_rule = True
  else:
      week52_span_rule = False
  screened_stocks[stock['Ticker']]['week52_span_rule'] = week52_span_rule
  
  # Close above 52 week high - 25%
  if prev_close > 0.75*week52_high:
      close_above_52weekhigh_rule = True
  else:
      close_above_52weekhigh_rule = False
  screened_stocks[stock['Ticker']]['close_above_52weekhigh_rule'] = close_above_52weekhigh_rule
  return screened_stocks

screened_stocks = {}
results = process_map(screen_stock, stock_list, max_workers=16)



    


In [61]:
screened_stocks = {}
for d in results:
  if d is not None:
    screened_stocks.update(d)
  
output_list = []
for stock in screened_stocks.keys():
    cols = ["Ticker"] + list(screened_stocks[stock].keys())
    temp_list = []
    temp_list.append(stock)
    for rule in screened_stocks[stock].keys():
        temp_list.append(screened_stocks[stock][rule])
    output_list.append(temp_list)
        
df_out = pd.DataFrame(output_list,columns=cols)

In [62]:
primary_rules = ['eps_QoQ_yearly_rule', 'sales_QoQ_yearly_rule', 'SMA200_slope_rule', 
                 'SMA150_greater_SMA200_rule', 'SMA50_greater_SMA150_rule', 'close_greater_SMA50_rule', 
                 'week52_span_rule', 'close_above_52weekhigh_rule']

secondary_rules = ['vol_price_rule', 'inst_ownership_rule', 'shares_outstanding_rule']
df_out['Primary Passed Tests'] = (df_out[list(primary_rules)]).sum(1)
df_out['Secondary Passed Tests'] = (df_out[list(secondary_rules)]).sum(1)

cols = df_out.columns.tolist()

for rule in primary_rules:
  if rule in cols:
    idx = cols.index(rule)
    cols[idx] = rule + " (1st)"

for rule in secondary_rules:
  if rule in cols:
    idx = cols.index(rule)
    cols[idx] = rule + " (2nd)"

df_out.columns = cols

In [63]:
df_out.sort_values('Primary Passed Tests', ascending=False)

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,EPS_QoQ_percent,Sales_QoQ_percent,prev_close,week52_high,week52_low,Inst. Ownership,Shares Outstanding,volume,vol_price_rule (2nd),eps_QoQ_yearly_rule (1st),sales_QoQ_yearly_rule (1st),shares_outstanding_rule (2nd),inst_ownership_rule (2nd),SMA200_slope_rule (1st),SMA150_greater_SMA200_rule (1st),SMA50_greater_SMA150_rule (1st),close_greater_SMA50_rule (1st),week52_span_rule (1st),close_above_52weekhigh_rule (1st),Primary Passed Tests,Secondary Passed Tests
201,AMRS,7.337591,8.860098,16.198000,2.5047,0.5532,0.252,0.968,20.82,22.340000,1.890000,0.383,246820000.0,1790295.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0
835,YETI,62.278613,67.000392,71.112000,0.3222,0.0501,12.158,0.263,75.03,78.040001,15.580000,0.987,87110000.0,505735.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0
894,APPS,51.790000,58.417451,80.793714,1.1260,0.1606,3.245,1.461,81.00,94.739998,3.830000,0.650,89000000.0,1961499.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0
1141,LSTR,139.872700,143.486960,158.032857,0.2283,0.0484,0.337,0.302,165.01,166.410004,86.769997,0.000,38390000.0,66988.0,False,True,True,False,False,True,True,True,True,True,True,8.0,0.0
534,PLT,26.651752,31.106275,40.653715,0.8311,0.1251,1.242,0.261,41.93,44.720001,7.440000,0.985,41250000.0,146341.0,False,True,True,False,False,True,True,True,True,True,True,8.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,STRA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0
411,RVNC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0
936,FN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0
410,HFWA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,None,None,None,0.0,0.0


In [64]:
df_out[df_out["Primary Passed Tests"] >= 8]

,Ticker,SMA200_value,SMA150_value,SMA50_value,SMA200_percent,SMA50_percent,EPS_QoQ_percent,Sales_QoQ_percent,prev_close,week52_high,week52_low,Inst. Ownership,Shares Outstanding,volume,vol_price_rule (2nd),eps_QoQ_yearly_rule (1st),sales_QoQ_yearly_rule (1st),shares_outstanding_rule (2nd),inst_ownership_rule (2nd),SMA200_slope_rule (1st),SMA150_greater_SMA200_rule (1st),SMA50_greater_SMA150_rule (1st),close_greater_SMA50_rule (1st),week52_span_rule (1st),close_above_52weekhigh_rule (1st),Primary Passed Tests,Secondary Passed Tests
9,KOPN,3.846058,4.701078,9.374571,2.5489,0.4404,1.174,0.598,11.14,13.560000,0.250000,0.1840,8.169000e+07,4075021.0,True,True,True,False,True,True,True,True,True,True,True,8.0,2.0
98,ABR,13.733664,14.455118,15.967772,0.3061,0.0566,1.049,0.416,16.63,16.969999,4.060000,0.4550,1.199200e+08,581533.0,False,True,True,False,False,True,True,True,True,True,True,8.0,0.0
182,TGNA,14.742336,15.670196,18.390857,0.4908,0.1759,1.881,0.351,20.62,20.620001,10.120000,0.9140,2.199400e+08,554568.0,False,True,True,False,False,True,True,True,True,True,True,8.0,0.0
197,MFC,17.214015,18.216176,20.263428,0.3303,0.0904,0.473,0.635,21.72,21.840000,8.900000,0.5370,1.940000e+09,923727.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0
201,AMRS,7.337591,8.860098,16.198000,2.5047,0.5532,0.252,0.968,20.82,22.340000,1.890000,0.3830,2.468200e+08,1790295.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0
209,CHX,13.992336,15.823333,20.356000,0.7410,0.1485,1.689,1.881,22.39,25.219999,3.020000,0.9980,2.002900e+08,558211.0,False,True,True,False,False,True,True,True,True,True,True,8.0,0.0
338,VUZI,8.942117,10.498137,17.823143,2.9861,0.8258,0.686,1.100,28.81,28.809999,1.090000,0.2550,5.570000e+07,5652125.0,True,True,True,False,True,True,True,True,True,True,True,8.0,2.0
385,SMPL,25.719635,26.938872,30.361714,0.2878,0.0600,5.221,0.519,32.26,34.709999,14.690000,0.9940,9.554000e+07,179833.0,False,True,True,False,False,True,True,True,True,True,True,8.0,0.0
409,HOME,19.991869,20.960059,26.327886,0.9247,0.3035,4.093,0.475,33.08,33.080002,1.420000,0.9430,6.451000e+07,705005.0,True,True,True,False,False,True,True,True,True,True,True,8.0,1.0
437,EGOV,26.002883,27.849559,33.159571,0.3613,0.0745,0.778,0.621,33.85,35.139999,19.420000,0.9060,6.703000e+07,236495.0,False,True,True,False,False,True,True,True,True,True,True,8.0,0.0
